# Online trajectory generation from invariant descriptor


This notebook serves as an example of on-line adaptation of demonstrated trajectories in a shape-preserving way


In [ ]:
import numpy as np
import sys
sys.path.append('../invariants_py')
import invariants_py.data_handler as dh
import matplotlib.pyplot as plt
import invariants_py.reparameterization as reparam
import scipy.interpolate as ip
from invariants_py.opti_calculate_vector_invariants_position_mf import OCP_calc_pos
from invariants_py.opti_generate_position_from_vector_invariants import OCP_generate_pos
from IPython.display import clear_output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Load example trajectory data and reparameterize to arclength


In [ ]:
data_location = '../data/sine_wave.txt'

# Load data
trajectory,time = dh.read_pose_trajectory_from_txt(data_location)

# Reparameterize
pose,time_profile,arclength,nb_samples,stepsize = reparam.reparameterize_trajectory_arclength(trajectory)
arclength_n = arclength/arclength[-1]
trajectory = pose[:,0:3,3]


Visualize the trajectory

In [ ]:
# Plot trajectory
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
ax = plt.axes(projection='3d')
ax.view_init(elev=26, azim=140)
ax.plot(trajectory[:,0],trajectory[:,1],trajectory[:,2],'-')


## Calculate invariant model for demonstrated trajectory


Two steps:
1. first specify optimization problem symbolically
2. calculate invariants given measured trajectory

In [ ]:
# specify optimization problem symbolically
FS_calculation_problem = OCP_calc_pos(window_len=nb_samples, bool_unsigned_invariants = False, 
                                           w_pos = 1, w_deriv = (10**-5)*np.array([1.0, 1.0, 1.0]), 
                                           w_abs = (10**-6)*np.array([1.0, 1.0]))

# calculate invariants given measurements
invariants, calculate_trajectory, movingframes = FS_calculation_problem.calculate_invariants_global(trajectory,stepsize)
init_vals_calculate_trajectory = calculate_trajectory
init_vals_movingframes = movingframes


Visualize calculated trajectory and invariants

In [ ]:
# Plot trajectory
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')
ax = plt.axes(projection='3d')
ax.plot(trajectory[:,0],trajectory[:,1],trajectory[:,2],'.-')
ax.plot(calculate_trajectory[:,0],calculate_trajectory[:,1],calculate_trajectory[:,2],'.-')

# Plot invariants
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(10,3))
ax1.plot(arclength_n,invariants[:,0])
ax1.set_title('Velocity [m/-]')
ax1.set_xlabel('s [-]')
ax2.plot(arclength_n,invariants[:,1])
ax2.set_title('Curvature [rad/-]')
ax2.set_xlabel('s [-]')
ax3.plot(arclength_n,invariants[:,2])
ax3.set_title('Torsion [rad/-]')
ax3.set_xlabel('s [-]')



In [ ]:
movingframes[:,0,0]

In [ ]:
import json
initialvalues_frames = np.hstack((movingframes[:,0,0], movingframes[:,1,0], movingframes[:,2,0], movingframes[:,0,1], movingframes[:,1,1], movingframes[:,2,1], movingframes[:,0,2], movingframes[:,1,2], movingframes[:,2,2], calculate_trajectory[:,0], calculate_trajectory[:,1], calculate_trajectory[:,2]))
invariants_model = np.hstack((invariants[:,0], invariants[:,1], invariants[:,2]))
json.dumps({"initial_frames":initialvalues_frames.tolist(), "invariants_model":invariants_model.tolist()})


## Trajectory generation during execution

Goal is to generate a single new trajectory during execution towards a new target, assuming we know the current progress s0

In [ ]:
# Function that allows to re-sample the invariant model given the progress range [s0,s1] in which we want a trajectory
def interpolate_model_invariants(demo_invariants, progress_values):
    
    resampled_invariants = np.array([demo_invariants(i) for i in progress_values]) 
    new_stepsize = progress_values[1] - progress_values[0] 
    resampled_invariants[:,0] = resampled_invariants[:,0] *  (progress_values[-1] - progress_values[0])
    return resampled_invariants, new_stepsize


### Resample the invariant model for the new progress range [s0,s1]

In [ ]:
# Parameterize invariants as a spline
knots = np.concatenate(([arclength_n[0]],[arclength_n[0]],arclength_n,[arclength_n[-1]],[arclength_n[-1]]))
spline_model_trajectory = ip.BSpline(knots,invariants,3)

# Define new start progress s0
current_progress = 0.4
number_samples = 40
progress_values = np.linspace(current_progress, arclength_n[-1], number_samples)
model_invariants,new_stepsize = interpolate_model_invariants(spline_model_trajectory,progress_values)

# Plot invariants
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(10,3))
ax1.plot(arclength_n,invariants[:,0])
ax1.plot(progress_values,model_invariants[:,0],'r.')
ax1.set_title('Velocity [m/m]')
ax2.plot(arclength_n,invariants[:,1])
ax2.plot(progress_values,model_invariants[:,1],'r.')
ax2.set_title('Curvature [rad/m]')
ax3.plot(arclength_n,invariants[:,2])
ax3.plot(progress_values,model_invariants[:,2],'r.')
ax3.set_title('Torsion [rad/m]')

print('Assume the task is at s=0.4, resample invariants between s=0.4 and s=1.0:')


### Define new constraints and solve the optimization problem to retrieve the trajectory

In [ ]:
# new constraints
current_index = round(current_progress*len(trajectory))
p_obj_start = calculate_trajectory[current_index]
p_obj_end = calculate_trajectory[-1] - np.array([-0.2, 0.0, 0.0])
R_FS_start = movingframes[current_index]
R_FS_end = movingframes[-1]

# specify optimization problem symbolically
FS_online_generation_problem = OCP_generate_pos(window_len=number_samples,w_invars = 10**2*np.array([10**1, 1.0, 1.0]))

# Solve
calculate_trajectory = init_vals_calculate_trajectory
movingframes = init_vals_movingframes
new_invars, new_trajectory, new_movingframes = FS_online_generation_problem.generate_trajectory(U_demo = model_invariants, 
                                                                                                p_obj_init = calculate_trajectory, 
                                                                                                R_t_init = movingframes, 
                                                                                                R_t_start = R_FS_start, 
                                                                                                R_t_end = R_FS_end, 
                                                                                                p_obj_start = p_obj_start, 
                                                                                                p_obj_end = p_obj_end, 
                                                                                                step_size = new_stepsize)



### Visualize resulting trajectory and invariants

In [ ]:
# Plot trajectory
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(elev=26, azim=140)
ax.plot(trajectory[:,0],trajectory[:,1],trajectory[:,2],'b')
ax.plot(new_trajectory[:,0],new_trajectory[:,1],new_trajectory[:,2],'r')
ax.plot(new_trajectory[-1,0],new_trajectory[-1,1],new_trajectory[-1,2],'r*')
ax.plot(new_trajectory[0,0],new_trajectory[0,1],new_trajectory[0,2],'ro')

# Plot invariants
fig = plt.figure(figsize=(14,5))
plt.subplot(1,3,1)
plt.plot(progress_values,new_invars[:,0],'r')
plt.plot(arclength_n,invariants[:,0],'b')
plt.plot(0,0)
plt.title('Velocity [m/-]')
plt.subplot(1,3,2)
plt.plot(progress_values,(new_invars[:,1]),'r')
plt.plot(arclength_n,invariants[:,1],'b')
plt.plot(0,0)
plt.title('Curvature [rad/-]')
plt.subplot(1,3,3)
plt.plot(progress_values,(new_invars[:,2]),'r')
plt.plot(arclength_n,invariants[:,2],'b')
plt.plot(0,0)
plt.title('Torsion [rad/-]')
plt.show()


## Online trajectory generation

Continuously generate new trajectories during execution towards a changing target

In [ ]:
# Specify optimization problem symbolically
window_len = 20
FS_online_generation_problem = OCP_generate_pos(window_len=window_len,w_invars = 10**1*np.array([10**1, 1.0, 1.0]))

# Initializing loop
current_progress = 0.0
old_progress = 0.0
calculate_trajectory = init_vals_calculate_trajectory
movingframes = init_vals_movingframes

while current_progress <= 1.0:
    
    print(f"current progress = {current_progress}")

    # Resample invariants for current progress
    progress_values = np.linspace(current_progress, arclength_n[-1], window_len)
    model_invariants,new_stepsize = interpolate_model_invariants(spline_model_trajectory,progress_values)
    
    # Select new boundary constraints
    current_index = round( (current_progress - old_progress) * len(calculate_trajectory))
    #print(current_index)
    p_obj_start = calculate_trajectory[current_index]
    p_obj_end = trajectory[-1] - current_progress*np.array([-0.2, 0.0, 0.0])
    R_FS_start = movingframes[current_index] 
    R_FS_end = movingframes[-1] 

    # Calculate remaining trajectory
    new_invars, calculate_trajectory, movingframes = FS_online_generation_problem.generate_trajectory(U_demo = model_invariants, 
                                                                                                      p_obj_init = calculate_trajectory, 
                                                                                                      R_t_init = movingframes, R_t_start = R_FS_start, 
                                                                                                      R_t_end = R_FS_end, p_obj_start = p_obj_start, 
                                                                                                      p_obj_end = p_obj_end, step_size = new_stepsize)

    # Plot trajectory
    clear_output(wait=True)
    fig = plt.figure(figsize=(14,8))
    ax = fig.add_subplot(231, projection='3d')
    ax.view_init(elev=26, azim=140)
    ax.plot(trajectory[:,0],trajectory[:,1],trajectory[:,2],'b')
    ax.plot(calculate_trajectory[:,0],calculate_trajectory[:,1],calculate_trajectory[:,2],'r')
    ax.plot(calculate_trajectory[-1,0],calculate_trajectory[-1,1],calculate_trajectory[-1,2],'r*')
    ax.plot(calculate_trajectory[0,0],calculate_trajectory[0,1],calculate_trajectory[0,2],'ro')
    
    # Plot invariants
    plt.subplot(2,3,4)
    plt.plot(progress_values,new_invars[:,0],'r')
    plt.plot(arclength_n,invariants[:,0],'b')
    plt.plot(0,0)
    plt.title('velocity [m/m]')
    plt.subplot(2,3,5)
    plt.plot(progress_values,(new_invars[:,1]),'r')
    plt.plot(arclength_n,invariants[:,1],'b')
    plt.plot(0,0)
    plt.title('curvature [rad/m]')
    plt.subplot(2,3,6)
    plt.plot(progress_values,(new_invars[:,2]),'r')
    plt.plot(arclength_n,invariants[:,2],'b')
    plt.plot(0,0)
    plt.title('torsion [rad/m]')
    plt.show()
    
    # New progress
    old_progress = current_progress
    current_progress = old_progress + 1/window_len
    
  

## Future work

implement all existing optimization problems:
- *invariant types*: screw axis invariants and vector invariants
- *measurement data*: poses, screws, vectors
- *problem types*: invariants calculation, trajectory generation


